In [ ]:
from task2.data_man.meta_data import ConllItem
from typing import AnyStr, Dict
from task2.data_man.meta_data import _assign_ner_tags, extract_spans, get_wiki_knowledge, join_tokens, get_wiki_title_knowledge, get_wiki_title_google_type, get_wiki_entities
from task2.data_man.meta_data import read_conll_item_from_file
from task2.configuration.config import logging
from task2.configuration import config

In [ ]:
def analyze_coverage(conll_file: AnyStr, entity_vocab: Dict, type: AnyStr='ORG'):
    conll_items = read_conll_item_from_file(conll_file)
    hit_count = 0
    total_count = 0 + 1e-6
    for item in conll_items:
        gold_spans = extract_spans(item.labels)
        gold_entities = []
        gold_labels = []
        for k in gold_spans:
            if gold_spans[k] == 'O':
                continue
            gold_entities.append(join_tokens(item.tokens[k[0]:k[1]+1])[0])
            gold_labels.append(item.labels[k[0]][2:])
        #logging.info('{}'.format(join_tokens(tokens)[0]))    
        #logging.info('gold entities: {}'.format(gold_entities))
        #logging.info('gold labels: {}'.format(gold_labels))
        for i, entity in enumerate(gold_entities):
            if entity in entity_vocab:
                if gold_labels[i] == type:
                    hit_count += 1
                    total_count += 1
            else:
                if gold_labels[i] == type:
                    logging.info('{}'.format(join_tokens(item.tokens)[0]))    
                    logging.info('{}: {} is not in'.format(entity, gold_labels[i]))
                    total_count += 1
    return '{}%'.format(round(hit_count / total_count, 4) * 100)

In [ ]:
person_entities_vocab = get_wiki_entities(config.wiki_entity_data['person'])
coverage = analyze_coverage(config.validate_file['Chinese'], person_entities_vocab, type='OtherPER')
logging.info('coverage is {}'.format(coverage))